# Computation Theory

## Problem 1: Binary Words and Operations

### Implement the following functions in Python. Use numpy to ensure that all variables and values are treated as 32-bit integers. These functions are defined in the Secure Hash Standard.

In [ ]:
#imports

import numpy as np